In [1]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from sklearn import preprocessing

from sklearn.metrics import r2_score,mean_squared_error

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("diamonds_train.csv", index_col=0)
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23,6134
1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66,532
2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96,1103
3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53,600
4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14,4997
...,...,...,...,...,...,...,...,...,...,...
40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55,11708
40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79,619
40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32,1267
40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88,9965


In [3]:
def codi(df):
    '''
    Esta función da valores numéricos a las columnas cut, color y clarity
    '''
    cols = ['cut', 'color', 'clarity']
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype('str'))
    return df
codi(df)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.21,2,4,5,63.0,57.0,6.73,6.70,4.23,6134
1,0.28,4,0,7,64.0,56.0,4.14,4.17,2.66,532
2,0.42,3,2,4,61.2,58.0,4.86,4.82,2.96,1103
3,0.26,2,4,1,61.1,57.0,4.16,4.12,2.53,600
4,1.10,1,3,2,63.4,57.0,6.52,6.55,4.14,4997
...,...,...,...,...,...,...,...,...,...,...
40340,1.55,3,4,5,61.3,61.0,7.46,7.39,4.55,11708
40341,0.36,2,0,2,60.6,56.0,4.58,4.63,2.79,619
40342,0.57,4,5,5,62.2,55.0,5.33,5.34,3.32,1267
40343,1.01,4,2,1,59.6,62.0,6.47,6.56,3.88,9965


In [4]:
X = np.array(df[['carat','cut', 'color', 'clarity']])
y = np.array(df["price"])


In [5]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Grid to search for the best random tree model
def random_tree_grid(X,y):
    rfc = RandomForestRegressor(n_jobs=-1,max_features= 'sqrt'
                                 ,n_estimators=50, criterion = "mse") 
    
    param_grid = { 
        'n_estimators': [10,25,40,50,60,75,100],
        'min_samples_split': [5,25,50,100],
        'min_samples_leaf': [5,25,50,100],
        'max_features': ['auto', 'sqrt', 'log2']
    }
    
    CV_rfc = GridSearchCV(estimator=rfc,verbose=1, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X, y)
    
    rfc = RandomForestRegressor(n_jobs=-1,**CV_rfc.best_params_, criterion = "mse")
    rfc.fit(X,y)
    
    return rfc

# Improved grid to search for the best random tree model
def tuning_random_tree_grid(X,y):
    rfc = RandomForestRegressor(n_jobs=-1,max_features= 'sqrt'
                                 ,n_estimators=50, criterion = "mse") 
    
    param_grid = { 
        'max_depth': [25,50,100,None],
        'min_samples_split': [2,5,10],
        'min_samples_leaf': [1,3,5],
        'max_features': ['auto', 'sqrt'],
        'n_estimators': [50,250, 1000, 1240, 1500, 2000]
    }
 
    CV_rfc = GridSearchCV(estimator=rfc,verbose=3, param_grid=param_grid, cv= 6)
    CV_rfc.fit(X, y)
    
    rfc = RandomForestRegressor(n_jobs=-1,**CV_rfc.best_params_, criterion = "mse")
    rfc.fit(X,y)
    
    return rfc,[CV_rfc.best_params_]

# Grid to search for the best ElasticNet model
def l2_l1_grid(X,y):
    parametersGrid = {"max_iter": [1_000, 5_000],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0,1.0,0.1)}

    eNet = ElasticNet()
    grid = GridSearchCV(eNet, parametersGrid,verbose=1, scoring='r2', cv=5)
    grid.fit(X, y)
    
    l2l1 = ElasticNet(**grid.best_params_)
    l2l1.fit(X,y)
    return l2l1


# Grid to search for the best KNN model
def knn_grid(X,y):
    params = {'n_neighbors':[3,5,9,13,15,31,51,61,75],
              'weights':['uniform', 'distance']}

    grid = GridSearchCV(
     estimator=KNeighborsRegressor(),
     param_grid=params,
     verbose=1,
     scoring="neg_mean_squared_error",
     return_train_score=True
     )
    
    grid.fit(X,y)
    knn=KNeighborsRegressor(**grid.best_params_)
    knn.fit(X,y)
    return knn

In [6]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import r2_score,mean_squared_error

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

# Returns the linear regresion and its test rmse
def first_test(X,y):
    print("LinearRegression")
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=153 )
    linreg = linear_model.LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_train)
    
    y_pred = linreg.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("RMSE (test) =",rmse)
    return linreg, rmse

# Returns the best Random Forest model and its test rmse
def forest_fit(X,y):
    print("Forest")
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=153)
    func = random_tree_grid(X_train, y_train)
    y_pred = func.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("RMSE (test) =",rmse)
    return func, rmse

# Returns the best ElasticNet model and its test rmse
def l2l1_fit(X,y):
    print("L2 L1")
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=153)
    func = l2_l1_grid(X_train, y_train)
    y_pred = func.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("RMSE (test) =",rmse)
    return func, rmse

# Returns the best KNN model and its test rmse
def knn_fit(X,y):
    print("KNN")
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=153)
    func = knn_grid(X_train, y_train)
    y_pred = func.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("RMSE (test) =",rmse)
    return func, rmse



# Improved function to obtain the best fitting Random Forest for a greater grid
def forest_tuning_fit(X,y):
    print("Forest")
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=153)
    func,bf = tuning_random_tree_grid(X_train, y_train)
    y_pred = func.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("RMSE (test) =",rmse)
    return func, rmse,bf

# Function to obtain the best fitting ML method ant its test RMSE
def best_func(X,y):
    best_one = None
    best_rmse = 1_000_000
    #lambda a,b: l2l1_fit(a,b),
    f_list=[lambda a,b: first_test(a,b),
            lambda a,b: knn_fit(a,b),
            lambda a,b: forest_fit(a,b) ]
    
    for func in f_list:
        fnc,rmse = func(X,y)
        if rmse < best_rmse:
            print("Choosen one best",rmse, best_rmse,rmse < best_rmse)
            best_rmse=rmse
            best_one=fnc
    
    return best_one,best_rmse

In [7]:
#Entrenamientos
first_test(X,y)
l2l1_fit(X,y)
forest_fit(X,y)
knn_fit(X,y)
forest_tuning_fit(X,y)
best_func(X,y)

LinearRegression
RMSE (test) = 1363.1867503555823
L2 L1
Fitting 5 folds for each of 140 candidates, totalling 700 fits
C:\Users\marin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26666402935.555344, tolerance: 43857672.21343696
  positive)
C:\Users\marin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26862124014.386856, tolerance: 43837395.2848854
  positive)
C:\Users\marin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26714252404.50979, tolerance: 43466785.02641014
  posit

In [ ]:
X_pred = pd.read_csv("diamonds_test.csv", index_col=0)
X_pred.head()

In [ ]:
codi(X_pred)

In [ ]:
pred_x = np.array(X_pred[['carat','cut', 'color', 'clarity']])

In [ ]:
predictions_submit = model.predict(pred_x)
predictions_submit

In [ ]:
sample = pd.read_csv("sample_submission.csv")
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [ ]:
from day4_competi_diamond.confeti_kaggle_apr import chequeator
chequeator(submission)

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np


# Grid to search for the best random tree model
def random_tree_grid(X,y):
    rfc = RandomForestRegressor(n_jobs=-1,max_features= 'sqrt'
                                 ,n_estimators=50, criterion = "mse") 
    
    param_grid = { 
        'n_estimators': [10,25,40,50,60,75,100],
        'min_samples_split': [5,25,50,100],
        'min_samples_leaf': [5,25,50,100],
        'max_features': ['auto', 'sqrt', 'log2']
    }
    
    CV_rfc = GridSearchCV(estimator=rfc,verbose=1, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X, y)
    
    rfc = RandomForestRegressor(n_jobs=-1,**CV_rfc.best_params_, criterion = "mse")
    rfc.fit(X,y)
    
    return rfc

# Improved grid to search for the best random tree model
def tuning_random_tree_grid(X,y):
    rfc = RandomForestRegressor(n_jobs=-1,max_features= 'sqrt'
                                 ,n_estimators=50, criterion = "mse") 
    
    param_grid = { 
        'max_depth': [25,50,100,None],
        'min_samples_split': [2,5,10],
        'min_samples_leaf': [1,3,5],
        'max_features': ['auto', 'sqrt'],
        'n_estimators': [50,250, 1000, 1240, 1500, 2000]
    }
 
    CV_rfc = GridSearchCV(estimator=rfc,verbose=3, param_grid=param_grid, cv= 6)
    CV_rfc.fit(X, y)
    
    rfc = RandomForestRegressor(n_jobs=-1,**CV_rfc.best_params_, criterion = "mse")
    rfc.fit(X,y)
    
    return rfc,[CV_rfc.best_params_]

# Grid to search for the best ElasticNet model
def l2_l1_grid(X,y):
    parametersGrid = {"max_iter": [1_000, 5_000],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0,1.0,0.1)}

    eNet = ElasticNet()
    grid = GridSearchCV(eNet, parametersGrid,verbose=1, scoring='r2', cv=5)
    grid.fit(X, y)
    
    l2l1 = ElasticNet(**grid.best_params_)
    l2l1.fit(X,y)
    return l2l1


# Grid to search for the best KNN model
def knn_grid(X,y):
    params = {'n_neighbors':[3,5,9,13,15,31,51,61,75],
              'weights':['uniform', 'distance']}

    grid = GridSearchCV(
     estimator=KNeighborsRegressor(),
     param_grid=params,
     verbose=1,
     scoring="neg_mean_squared_error",
     return_train_score=True
     )
    
    grid.fit(X,y)
    knn=KNeighborsRegressor(**grid.best_params_)
    knn.fit(X,y)
    return knn